In [24]:
import BioSimSpace as BSS
import pandas as pd
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

In [37]:
def getPertSys(testset, ref_ligand, protein):
    for index, row in testset.iterrows():
        title = row['Molecule Name']
        path = "./pertSystems/" + title
        
        if os.path.exists(path):
            print("Perturtable system for " + title + "exists")
        else:
            # Parametrise the ligand 
            print("Parameterising " + title +  " ...")
            ligand = BSS.IO.readMolecules('./input/ligands/' + title + '.pdb')[0]    
            ligand = BSS.Parameters.gaff2(ligand, work_dir = './lig_par/' + title).getMolecule()
    
            # Morph and combine systems
            mapping = BSS.Align.matchAtoms(ligand, ref_ligand)
            ligand = BSS.Align.rmsdAlign(ligand, ref_ligand, mapping)
            merged = BSS.Align.merge(ligand, ref_ligand, mapping)
            complx = merged + protein 
    
            print("Solvating " + title + " ...")
            # Solvate the protein ligand complex and merged ligand in water
            complx_sol = BSS.Solvent.tip3p(molecule = complx, box = 3*[90*BSS.Units.Length.angstrom])
            merged_sol = BSS.Solvent.tip3p(molecule = merged, box = 3*[40*BSS.Units.Length.angstrom])

            print("Minimising " + title + " ...")
            # Create the minimisation protocol
            prot_minimisation = BSS.Protocol.Minimisation(steps = 2000)
            # Minimise both legs
            proc_minimisation_complx = BSS.MD.run(complx_sol, 
                                                  prot_minimisation, 
                                                  work_dir = './minimisation/' + title + '/complex')
            complx_minimised = proc_minimisation_complx.getSystem(block = True)
            proc_minimisation_merged = BSS.MD.run(merged_sol, 
                                                  prot_minimisation, 
                                                  work_dir = './minimisation/' + title +'/merged')
            merged_minimised = proc_minimisation_merged.getSystem(block = True)

            print("Equilibrating " + title + " ...")
            # Create the equilibration protocol
            prot_equilibration = BSS.Protocol.Equilibration(timestep = 2*BSS.Units.Time.femtosecond,
                                                            runtime = 500*BSS.Units.Time.picosecond)
            # Equilibrate both legs
            proc_equilibration_complx = BSS.MD.run(complx_minimised,
                                                   prot_equilibration,
                                                   work_dir = './equilibration/' + title + '/complex')
            complx_equilibrated = proc_equilibration_complx.getSystem(block = True)
            proc_equilibration_merged = BSS.MD.run(merged_minimised, 
                                                   prot_equilibration,
                                                   work_dir = './equilibration/' + title + '/merged')
            merged_equilibrated = proc_equilibration_merged.getSystem(block = True) 
            
            BSS.IO.savePerturbableSystem(path + '/' + title + '_bound', complx_equilibrated)
            BSS.IO.savePerturbableSystem(path + '/' + title + '_free', merged_equilibrated)
            print("Perturbable systems for " + title + " saved")

In [40]:
def getFreeEnergy(title, lam_bound, lam_free):
    # Read perturbable system
    path = "./pertSystems/" + title + '/'
    boundSystem = BSS.IO.readPerturbableSystem(path + title + '_bound0.prm7', path + title + '_bound0.rst7',
                                                path + title + '_bound1.prm7', path + title + '_bound1.rst7')
    freeSystem = BSS.IO.readPerturbableSystem(path + title + '_free0.prm7', path + title + '_free0.rst7',
                                                path + title + '_free1.prm7', path + title + '_free1.rst7')
    
    # Create the free energy protocols
    prot_bound = BSS.Protocol.FreeEnergy(timestep = 2*BSS.Units.Time.femtosecond,
                                         runtime = 200*BSS.Units.Time.picosecond,
                                         num_lam = lam_bound)
    prot_free = BSS.Protocol.FreeEnergy(timestep = 2*BSS.Units.Time.femtosecond,
                                        runtime = 200*BSS.Units.Time.picosecond,
                                        num_lam = lam_free)
    
    # Initialise the free energy object for each leg
    fep_bound = BSS.FreeEnergy.Relative(boundSystem,
                                        prot_bound,
                                        engine = "somd",
                                        work_dir = './somd/' + title + "/bound")
    fep_free = BSS.FreeEnergy.Relative(freeSystem,
                                       prot_free,
                                       engine = "somd",
                                       work_dir = './somd/' + title + "/free")
    
    # Run FEP
    print("Computing FEP for " + title + " bound leg ...")
    fep_bound.run()
    fep_bound.wait()
    print("Computing FEP for " + title + " free leg ...")
    fep_free.run()
    fep_free.wait()
    
    # Get the PMF and the overlap matrix
    pmf_bound, overlap_bound = fep_bound.analyse()
    pmf_free, overlap_free = fep_free.analyse()
    
    # Compute the relative free-energy difference with PMF
    binding_free_energy = BSS.FreeEnergy.Relative.difference(pmf_bound, pmf_free)
    
    return binding_free_energy, overlap_bound, overlap_free

In [27]:
# Plot and save heatmaps of overlap matrixes

def getHeatMap(bound, free, title):
    fig, axs = plt.subplots(ncols = 2, nrows = 1, figsize = (20, 10), constrained_layout = True)
    fig.supxlabel('λ index',fontsize = 14)
    fig.supylabel('λ index',fontsize = 14)
    
    cmap = mpl.cm.viridis
    bounds = [0, 0.1, 0.2, 0.4, 0.6, 0.8]
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    fig.colorbar(mpl.cm.ScalarMappable(norm = norm, cmap = cmap))
    
    for col in range(2):
        if col == 0:
            leg = "Bound leg"
            matrix = np.array(bound)
        else:
            leg = "Free leg"
            matrix = np.array(free)
        lamb = matrix.shape[0]

        im = axs[col].imshow(matrix, norm = norm, cmap = cmap)   
        
        axs[col].set_xticks(np.arange(lamb), labels = np.arange(lamb))
        axs[col].set_yticks(np.arange(lamb), labels = np.arange(lamb))
        
        axs[col].set_title(leg, fontsize=16)
        axs[col].spines[:].set_visible(False)
        axs[col].grid(visible = True, which = "minor", axis = 'both', color = "w", linestyle = '-', linewidth = 3)
        
        for i in range(lamb):
            for j in range(lamb):
                text = axs[col].text(j, i, '{:.2f}'.format(matrix[i, j]), ha = "center", va = "center", color = "w", fontsize = 12)
    
    fig.savefig('overlap_heatmaps/' + title)

In [28]:
# Get perturbable systems

testset = pd.read_csv("./input/testset.csv")[1:3]
if not os.path.exists('overlap_heatmaps'): os.makedirs('overlap_heatmaps')
    
lam_bound = 15
lam_free = 10

# Parameterise the reference ligand
ref_id = 'CVD-0007756.pdb'
if os.path.exists("./" + ref_id):
    print("Parameterised reference ligand exists")
    ref_ligand = BSS.IO.readMolecules(['./' + ref_id + './' + ref_id + '.prm7', './' + ref_id + './' + ref_id + '.rst7'])
else:
    print("Parameterising the reference ligand ...")
    ref_ligand = BSS.Parameters.gaff2(BSS.IO.readMolecules('./input/' + ref_id)[0], work_dir = "./reflig_par").getMolecule()
    BSS.IO.saveMolecules("./" + ref_id + '/' + ref_id, ref_ligand, ["PRM7", "RST7"])

# Add missing hydrogens and parameterise the protein
if os.path.exists("./protein"):
    print("Parameterised protein exists")
    protein = BSS.IO.readMolecules(['./protein/protein.prm7', './protein/protein.rst7'])    
else:
    print("Parameterising the protein ...")
    try:
        protein = BSS.Parameters.ff14SB(BSS.IO.readMolecules('./input/protein_noH.pdb')[0], work_dir = "./protein_par").getMolecule()
    except:
        protein = BSS.Parameters.ff14SB(BSS.IO.readMolecules(['./protein_par/leap.crd','./protein_par/leap.top'])[0], work_dir = "./protein_par_1").getMolecule()
        BSS.IO.saveMolecules("./protein/protein", protein, ["PRM7", "RST7"])

getPertSys(testset, ref_ligand, protein)

Parameterising the reference ligand
Parameterising the protein
Parameterising CVD-0012808 ...
Solvating CVD-0012808 ...
Minimising CVD-0012808 ...
Equilibrating CVD-0012808 ...
Perturbable systems for CVD-0012808 saved
Parameterising CVD-0015217 ...
Solvating CVD-0015217 ...
Minimising CVD-0015217 ...
Equilibrating CVD-0015217 ...
Perturbable systems for CVD-0015217 saved


TypeError: readPerturbableSystem() missing 3 required positional arguments: 'coords0', 'top1', and 'coords1'

In [ ]:
# Run FEPs and save results

result = open('result.txt', 'w')
for index, row in testset.iterrows():
    title = row['Molecule Name']
    FreeEnergy, overlap_bound, overlap_free = getFreeEnergy(title, lam_bound, lam_free)
    getHeatMap(overlap_bound, overlap_free, title)
    result.writelines(title + ' ' + str(FreeEnergy[0]) + ' ' + str(FreeEnergy[1]) + '\n')
result.close()

Computing FEP for CVD-0012808bound leg
